# VacationPy (Part 2)
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load files
cities_file = "output_data/cities.csv"

# Read school data file and store into Pandas DataFrames
cities_df = pd.read_csv(cities_file)

cities_df.head()

,City_ID,City,Country,Latitude,Longitude,Date,Max Temperature (C),Humidity,Cloudiness,Wind Speed (km/h)
0,0,Tiksi,RU,71.6872,128.8694,2021-03-19 07:07:42,-18.92,98,100,3.67
1,1,New Norfolk,AU,-42.7826,147.0587,2021-03-19 07:05:30,21.11,47,0,1.34
2,2,Klaksvík,FO,62.2266,-6.5890,2021-03-19 07:07:42,7.00,100,75,7.20
3,3,Korla,CN,41.7597,86.1469,2021-03-19 07:07:42,3.88,72,45,3.78
4,4,Rikitea,PF,-23.1203,-134.9692,2021-03-19 07:01:32,26.78,78,100,7.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

#Store humidity levels
humidity = cities_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(4,17), zoom_level=1.9,layout={'width': '965px', 'height': '800px'})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='800px', width='965px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.


* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
# Conditions
min_temp = 18
max_temp = 32
max_wind = 10
cloud_level = 0

vacation_df = cities_df.loc[(cities_df["Max Temperature (C)"]<=max_temp) 
                            & (cities_df["Max Temperature (C)"]>=min_temp)
                            & (cities_df["Cloudiness"]==cloud_level)
                            & (cities_df["Wind Speed (km/h)"]<=max_wind),:]
vacation_df.head()
 

,City_ID,City,Country,Latitude,Longitude,Date,Max Temperature (C),Humidity,Cloudiness,Wind Speed (km/h)
1,1,New Norfolk,AU,-42.7826,147.0587,2021-03-19 07:05:30,21.11,47,0,1.34
55,55,Esperance,AU,-33.8667,121.9000,2021-03-19 07:07:53,31.00,23,0,1.54
95,95,Acapulco de Juárez,MX,16.8634,-99.8901,2021-03-19 07:08:04,22.89,79,0,1.80
105,105,Bilma,NE,18.6853,12.9164,2021-03-19 07:08:06,29.86,14,0,1.43
106,106,Port Lincoln,AU,-34.7333,135.8667,2021-03-19 07:08:06,23.24,58,0,5.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
# Target city
coord = "43.6139805,-116.2037792"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'latlng={0}&key={1}').format(coord, g_key)
# Run a request to endpoint and convert result to json
geo_data = requests.get(target_url).json()

# Print the json
pprint(geo_data)

{'plus_code': {'compound_code': 'JQ7W+HF Boise, ID, USA',
               'global_code': '85M5JQ7W+HF'},
 'results': [{'address_components': [{'long_name': '245',
                                      'short_name': '245',
                                      'types': ['street_number']},
                                     {'long_name': 'South Capitol Boulevard',
                                      'short_name': 'S Capitol Blvd',
                                      'types': ['route']},
                                     {'long_name': 'Downtown',
                                      'short_name': 'Downtown',
                                      'types': ['neighborhood', 'political']},
                                     {'long_name': 'Boise',
                                      'short_name': 'Boise',
                                      'types': ['locality', 'political']},
                                     {'long_name': 'Ada County',
                                      

In [50]:
# Print the name and address of the first restaurant that appears
pprint(geo_data["results"][0]["address_components"][0])

{'long_name': '245', 'short_name': '245', 'types': ['street_number']}


In [43]:
# geocoordinates
latitude=vacation_df["Latitude"].tolist()
longitude=vacation_df["Longitude"].tolist()


# set up lists to hold reponse info
hotelname = []
lat = []
lon = []

# Loop through the list of cities and perform a request for data on each
for x,y in zip(latitude,longitude):
    
    target_search = "Hotel"
    target_radius = 5000
    
    # set up a parameters dictionary
    params = {
    "location": f"{x},{y}",
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        hotelname.append(response["results"][0]["name"])
        lat.append(response["results"][0]["geometry"]["location"]["lat"])
        lon.append(response["results"][0]["geometry"]["location"]["lng"]) 
        
    except:
       # Append null values
       print(f"({x},{y}) not found. Skipping...")
    pass


(18.6853,12.9164) not found. Skipping...
(3.0707,172.7902) not found. Skipping...
(22.9036,-83.1592) not found. Skipping...
(3.0707,172.7902) not found. Skipping...
(24.1447,67.4497) not found. Skipping...
(3.0707,172.7902) not found. Skipping...
(3.0707,172.7902) not found. Skipping...
(3.0707,172.7902) not found. Skipping...
(21.1167,70.1167) not found. Skipping...
(20.1986,1.0114) not found. Skipping...


In [44]:
print(hotelname)

['The Woodbridge', 'Esperance Chalet Village', 'Hotel Las Brisas Acapulco', 'Marina Hotel', 'The Fern Sattva Resort Dwarka', 'The Woodbridge', 'Marina Star Hotel', 'Hôtel Temet', 'Alqadri hotel zero point ormara', 'The Woodbridge', 'The Woodbridge', 'Codrington Gardens Bed And Breakfast', 'Codrington Gardens Bed And Breakfast', 'The Woodbridge', 'The Woodbridge', 'Hotel Miramar Fogo Brava', 'Hotel Miramar Fogo Brava', 'PiscoMar Peru', 'ROYAL hotel e gastronomia', 'Hotel Ami Grace', 'Star Emirates garden villas', 'Codrington Gardens Bed And Breakfast', 'The Old Mount Gambier Gaol', 'Esperance Chalet Village', 'PiscoMar Peru', 'Honeylocust', 'PiscoMar Peru', 'Hotel Las Brisas Acapulco', 'Esperance Chalet Village', 'Saldanha Bay Hotel', 'Rydges Adelaide', 'Esperance Chalet Village', 'Big Game Maio', 'Shreemati Khetbai Ramji Momaya Gala Senotorium', 'LUX Grand Gaube Resort & Villas', 'Saldanha Bay Hotel', 'Hôtel Temet', 'Esperance Chalet Village']


In [ ]:
# create a data frame from cities, lat, and temp
hotel_df = pd.DataFrame({"Hotel Name": hotelname,
                            "City": name,
                            "Country": country,
                            "Lat": lat,
                            "Lng": lon
                            })

hotel_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
